## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.80,-68.30,46.40,75,75,23.04,shower rain
1,1,Glendive,US,47.11,-104.71,62.60,36,1,9.17,clear sky
2,2,Busselton,AU,-33.65,115.33,61.00,51,0,15.90,clear sky
3,3,Tuktoyaktuk,CA,69.45,-133.04,3.20,71,20,14.99,few clouds
4,4,Rikitea,PF,-23.12,-134.97,74.55,69,20,12.71,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Glendive,US,47.11,-104.71,62.60,36,1,9.17,clear sky
2,2,Busselton,AU,-33.65,115.33,61.00,51,0,15.90,clear sky
4,4,Rikitea,PF,-23.12,-134.97,74.55,69,20,12.71,few clouds
5,5,Saldanha,ZA,-33.01,17.94,60.80,87,100,8.05,overcast clouds
6,6,Almunecar,ES,36.73,-3.69,66.99,39,100,1.99,overcast clouds
8,8,Avarua,CK,-21.21,-159.78,75.20,78,73,6.93,light rain
9,9,Cape Town,ZA,-33.93,18.42,66.99,87,40,19.46,scattered clouds
10,10,Mahebourg,MU,-20.41,57.70,73.40,78,40,8.05,scattered clouds
11,11,Albany,US,42.60,-73.97,60.80,44,75,8.05,broken clouds
13,13,Hobart,AU,-42.88,147.33,57.99,50,75,13.87,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                389
City                   389
Country                383
Lat                    389
Lng                    389
Max Temp               389
Humidity               389
Cloudiness             389
Wind Speed             389
Current Description    389
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

clean_df.count()

City_ID                383
City                   383
Country                383
Lat                    383
Lng                    383
Max Temp               383
Humidity               383
Cloudiness             383
Wind Speed             383
Current Description    383
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Glendive,US,62.60,clear sky,47.11,-104.71,
2,Busselton,AU,61.00,clear sky,-33.65,115.33,
4,Rikitea,PF,74.55,few clouds,-23.12,-134.97,
5,Saldanha,ZA,60.80,overcast clouds,-33.01,17.94,
6,Almunecar,ES,66.99,overcast clouds,36.73,-3.69,
8,Avarua,CK,75.20,light rain,-21.21,-159.78,
9,Cape Town,ZA,66.99,scattered clouds,-33.93,18.42,
10,Mahebourg,MU,73.40,scattered clouds,-20.41,57.70,
11,Albany,US,60.80,broken clouds,42.60,-73.97,
13,Hobart,AU,57.99,broken clouds,-42.88,147.33,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

clean_hotel_df = hotel_df.dropna(axis=0, how='any', thresh=None, subset=["Hotel Name"], inplace=False)
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Glendive,US,62.60,clear sky,47.11,-104.71,Baymont by Wyndham Glendive
2,Busselton,AU,61.00,clear sky,-33.65,115.33,Observatory Guest House
4,Rikitea,PF,74.55,few clouds,-23.12,-134.97,Pension Maro'i
5,Saldanha,ZA,60.80,overcast clouds,-33.01,17.94,Blue Bay Lodge
6,Almunecar,ES,66.99,overcast clouds,36.73,-3.69,Almuñécar Playa Spa Hotel
8,Avarua,CK,75.20,light rain,-21.21,-159.78,Paradise Inn
9,Cape Town,ZA,66.99,scattered clouds,-33.93,18.42,Southern Sun Waterfront Cape Town
10,Mahebourg,MU,73.40,scattered clouds,-20.41,57.70,Shandrani Beachcomber Resort & Spa
13,Hobart,AU,57.99,broken clouds,-42.88,147.33,St Ives Hobart Accommodation Tasmania
17,Wenling,CN,69.01,scattered clouds,28.37,121.36,Hanting Express Wenling Wanchang North Road


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))